Researching the impact of amount of dimensions for a certain epsilon.
Instead of having a linechart with the epsilons on the x-axis, we will show the dimensions for the heart dataset and seed dataset.
We do this only for the nd-Laplace algorithm.



In [4]:
from Helpers import helpers
# we select epsilon 3.0 for now
epsilon = 3
plain_df, perturbed_df = helpers.load_plain_and_perturbed_dataset(epsilon, perturbed_path='../ExperimentRunners/data/nd-laplace-truncated/heart-dataset/', import_path='../data/heart-dataset/heart_numerical.csv')
perturbed_df.head()


,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks
0,123.617236,-0.092564,-0.004802,0.000369,0.000040,64.000004,61.999999,126.000000,2.000000
1,134.624873,4.697431,0.118039,3.917153,1.983983,129.996073,67.998539,197.999988,6.000005
2,136.550383,1.619046,0.175764,4.997016,1.998522,130.000103,67.999957,197.999979,4.999995
3,137.572928,1.539276,0.057311,6.015456,2.011067,116.997654,52.998808,170.000397,11.000089
4,135.003936,4.689872,0.262001,4.996434,0.001184,116.999581,53.000079,169.999985,8.999989
